**Loading packages**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.max_columns', 100)

**Loading data**

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

**Data at first sight**

Here is an excerpt of the the data description for the competition:
다음은 경기에 대한 데이터 설명이다.

Features that belong to similar groupings are tagged as such in the feature names (e.g., ind, reg, car, calc).
유사한 그룹에 속하는 feature들은 feature 이름(예: ind, reg, car, calc)에서와 같이 태그 지정된다.

Feature names include the postfix bin to indicate binary features and cat to indicate categorical features.
feature 
이름에는 2진수 특징을 나타내는 사후 수정 bin과 범주형 특징을 나타내는 cat이 포함된다.

Features without these designations are either continuous or ordinal.
이러한 지정이 없는 형상은 연속 또는 순서형이다.

Values of -1 indicate that the feature was missing from the observation.
-1 값은 feature가이 관찰에서 누락되었음을 나타낸다.

The target columns signifies whether or not a claim was filed for that policy holder.
target 열은 해당 정책 보유자에 대한 클레임이 접수되었는지 여부를 나타낸다.

Ok, that's important information to get us started. Let's have a quick look at the first and last rows to confirm all of this.
좋아, 시작하는 데 중요한 정보야. 이 모든 것을 확인하기 위해 첫 번째 행과 마지막 행들을 간단히 살펴봅시다.

In [ ]:
train.head()

In [ ]:
train.tail()

We indeed see the following
우리는 정말로 다음을 본다.

* binary variables
이항 변수

* categorical variables of which the category values are integers
범주 값이 정수인 범주형 변수

* other variables with integer or float values
정수 또는 부동 값을 가진 다른 변수

* variables with -1 representing missing values
결측값을 나타내는 -1의 변수

* the target variable and an ID variable
target 변수 및 ID 변수

Let's look at the number of rows and columns in the train 
data.
train 데이터의 행과 열의 수를 살펴보자.

In [ ]:
train.shape

We have 59 variables and 595.212 rows.
59개의 변수와 595.212개의 행이 있다.

Let's see if we have the same number of variables in the test data.
테스트 데이터에 동일한 수의 변수가 있는지 확인하십시오.

Let's see if there are duplicate rows in the training data.
training 데이터에 중복된 행이 있는지 확인하십시오.

In [ ]:
train.drop_duplicates()
train.shape

No duplicate rows, so that's fine.
중복되는 행이 없으니 괜찮아.

In [ ]:
test.shape

We are missing one variable in the test set, but this is the target variable. So that's fine.
테스트 세트에 변수가 한 개 누락되어 있지만 대상 변수 입니다. 그래, 괜찮아.

Let's now invesigate how many variables of each type we have.
이제 각 유형의 변수가 몇 개나 있는지 살펴봅시다.

So later on we can create dummy variables for the 14 categorical variables.
그래서 나중에 14개의 범주형 변수에 대한 더미 변수를 만들 수 있다.

The bin variables are already binary and do not need dummification.
binary 변수는 이미 2진법이기 때문에 dummization이 필요하지 않다.

In [ ]:
train.info()

Again, with the info() method we see that the data type is integer or float. 
다시, info() 방법을 사용하여 데이터 유형이 정수 또는 부동임을 알 수 있다.

No null values are present in the data set. 
데이터 세트에 null 값이 없음.

That's normal because missing values are replaced by -1. 
결측값이 -1로 대체되기 때문에 정상이다.

We'll look into that later.
우리는 그것을 나중에 살펴볼 것이다.



**Metadata**

To facilitate the data management, we'll store meta-information about the variables in a DataFrame.
데이터 관리를 용이하게 하기 위해 변수에 대한 메타 정보를 DataFrame에 저장할 것이다.

This will be helpful when we want to select specific variables for analysis, visualization, modeling, ...
이는 분석, 시각화, 모델링 등을 위한 특정 변수를 선택하고자 할 때 도움이 될 것이다.

Concretely we will store:
구체적으로 저장:

* role: input, ID, target
* level: nominal(명목/범주형자료), interval(구간자료), ordinal(순서형자료), binary
http://blog.heartcount.io/dd
* keep: True or False
* dtype: int, float, str

In [ ]:
data = []
for f in train.columns:
    # defining the role
    if f=='target':
        role='target'
    elif f=='id':
        role='id'
    else:
        role='input'
    
    # defining the level
    if 'bin' in f or f == 'target':
        level = 'binary'
    elif 'cat' in f or f == 'id':
        level = 'nominal'
    elif train[f].dtype ==float:
        level = 'interval'
    elif train[f].dtype == int:
        level = 'ordinal'
        
    #Initialize keep to True for all variables except f or id
    keep = True
    if f =='id':
        keep = False
    
    # Defining the data type
    dtype = train[f].dtype
    
    #Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname' : f,
        'role' : role,
        'level' : level,
        'keep' : keep,
        'dtype' : dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns = ['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname',inplace = True)
    

In [ ]:
meta

Example to extract all nominal variables that are not dropped

삭제되지 않은 모든 nominal 변수를 추출하는 예제


In [ ]:
meta[(meta.level == 'nominal') & (meta.keep)].index

Below the number of variables per role and level are displayed.
역할 및 수준별 변수 수 아래에 표시된다.

In [ ]:
pd.DataFrame({'count':meta.groupby(['role','level'])['role'].size()}).reset_index()

**Descriptive statistics**

We can also apply the describe method on the dataframe. 
우리는 또한 데이터프레임에 설명법을 적용할 수 있다. 

However, it doesn't make much sense to calculate the mean, std, ... on categorical variables and the id variable. 
그러나 범주형 변수와 ID 변수에 대한 평균, std, ...을 계산하는 것은 그다지 이치에 맞지 않는다. 

We'll explore the categorical variables visually later.
우리는 나중에 시각적으로 범주형 변수를 탐구할 것이다.

Thanks to our meta file we can easily select the variables on which we want to compute the descriptive statistics.
메타 파일 덕분에 우리는 기술 통계량을 계산할 변수를 쉽게 선택할 수 있다.

To keep things clear, we'll do this per data type.
확실히 하기 위해, 우리는 데이터 유형별로 이것을 할 것이다.
 

**Interval variables**

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

reg variables
* only ps_reg_03 has missing values
ps_reg_03에만 결측값이 있음

* the range (min to max) differs between the variables. We could apply scaling (e.g. StandardScaler), but it depends on the classifier we will want to use.
변수마다 범위(min to max)가 다르다. **스케일링(예: StandardScaler)을 적용할 수 있지만, 사용할 분류기에 따라 달라진다.**


car variables
* ps_car_12 and ps_car_15 have missing values
ps_car_12 및 ps_car_15에 결측값이 있음

* again, the range differs and we could apply scaling.
다시 한 번, 그 범위는 달라졌고, 우리는 스케일링을 적용할 수 있었다.


calc variables
* no missing values
결측값 없음

* this seems to be some kind of ratio as the maximum is 0.9
이것은 최대치가 0.9이기 때문에 일종의 비율인 것 같다.

* all three _calc variables have very similar distributions
세 가지 _calc 변수는 모두 매우 유사한 분포를 가지고 있다.


Overall, we can see that the range of the interval variables is rather small. 
전체적으로, 우리는 간격 변수의 범위가 다소 작다는 것을 알 수 있다. 
Perhaps some transformation (e.g. log) is already applied in order to anonymize the data?
데이터를 익명화하기 위해 일부 변환(예: 로그)이 이미 적용되었을 수 있는가?





**Ordinal variables**

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
train[v].describe()

* Only one missing variable: ps_car_11
누락된 변수만: ps_car_11

* We could apply scaling to deal with the different ranges
다양한 범위를 처리하기 위해 스케일링을 적용할 수 있다.



**Binary variables**

In [ ]:
v = meta[(meta.level == 'binary') & (meta.keep)].index
train[v].describe()

* ** A priori in the train data is 3.645%, which is strongly imbalanced.
train 데이터의 선취율은 3.645%로 매우 불균형적이다.**

* From the means we can conclude that for most variables the value is zero in most cases.
mean으로부터 우리는 대부분의 변수에 대한 값이 0이라고 결론을 내릴 수 있다.

**Handling imbalanced classes**

As we mentioned above the proportion of records with target=1 is far less than target=0.
위에서 언급한 바와 같이, target=1을 갖는 기록의 비율은 target=0보다 훨씬 낮다.

This can lead to a model that has great accuracy but does have any added value in practice. 
따라서 정확도가 높지만 실제로 가치가 있는 모델이 될 수 있다.

Two possible strategies to deal with this problem are:
이 문제를 해결하기 위한 두 가지 가능한 전략은 다음과 같다.

* oversampling records with target=1
target=1로 레코드 **오버샘플링**

* undersampling records with target=0
target=0으로 레코드 **언더샘플링**

There are many more strategies of course and MachineLearningMastery.com gives a nice overview.
물론 더 많은 전략들이 있고 MachineLearningMastery.com은 멋진 개요를 제공한다.
As we have a rather large training set, we can go for undersampling.
우리는 꽤 큰 훈련 세트를 가지고 있기 때문에, 언더샘플링을 할 수 있다.


96 ->0
4 -> 1

* 오버샘플링은 4를 늘려서 50 부풀린 다음 ->1
총 146개의 샘플이 생기는 것.

* 언더샘플링은 96개를 50개 ->0
4->1
54개의 샘플이 생기는 것.

**smote**
캐글코리아에 공유되어있음..! imbalanced-learn 라이브러리

Q. 명목형변수(categorical var) smote 사용할수 있을까?

In [ ]:
desired_apriori=0.10

# Get the indices per target value
idx_0 = train[train.target==0].index
idx_1 = train[train.target==1].index

# Get original number of records per target value
nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

# Calculate the undersampling rate and resulting number of records with target =0
undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('Rate to undersample records with target = 0 : {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# Randomly select records with target=0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state=37, n_samples = undersampled_nb_0)
#몇개 뽑을래 하니깐 195246만큼.
# Construct list with remaining indices
idx_list = list(undersampled_idx) + list(idx_1)

# Return undersample data frame
train = train.loc[idx_list].reset_index(drop=True)
#셔플했으니 리셋인덱스 해준다.

In [ ]:
print(idx_list)

**Data Quality Checks**

Checking missing values

Missings are represented as -1

In [ ]:
vars_with_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings>0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missings values'.format(f, missings, missings_perc))

print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))        
    

* **ps_car_03_cat and ps_car_05_cat** have a large proportion of records with missing values. Remove these variables.
ps_car_03_cat와 ps_car_05_cat은 결측값을 가진 기록의 많은 부분을 가지고 있다. 이러한 변수를 제거하십시오.

* For the other categorical variables with missing values, we can leave the missing value -1 as such.
결측값이 있는 다른 범주형 변수의 경우 결측값 -1을 그대로 남겨둘 수 있다.

* **ps_reg_03** (continuous) has missing values for 18% of all records. Replace by the mean.
ps_reg_03(연속)은 모든 기록의 18%에 대한 결측값을 가지고 있다. 평균으로 대체한다.

* **ps_car_11** (ordinal) has only 5 records with misisng values. Replace by the mode.
ps_car_11(주문)에는 오수 값이 있는 레코드가 5개뿐입니다. 모드(최빈값)로 교체한다.

* **ps_car_12** (continuous) has only 1 records with missing value. Replace by the mean.
ps_car_12(연속)에는 결측값이 있는 레코드가 1개만 있다. 평균으로 대체한다.

* **ps_car_14** (continuous) has missing values for 7% of all records. Replace by the mean.
ps_car_14(연속)는 모든 기록의 7%에 대한 결측값을 가지고 있다. 평균으로 대체한다.


In [ ]:
# Dropping the variables with too many missing values 
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace=True,axis=1)
meta.loc[(vars_to_drop), 'keep'] = False #Updating the meta

#Imputing with mean or mode
mean_imp = Imputer(missing_values=-1,strategy='mean', axis=0)
mode_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)

train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()

**Checking the cardinality of the categorical variables**

Cardinality refers to the number of different values in a variable. 
카디널리티는 변수에 있는 다른 값의 수를 의미한다.

As we will create dummy variables from the categorical variables later on, we need to check whether there are variables with many distinct values.
추후 범주형 변수에서 더미 변수를 만들 예정이므로 구별되는 값이 많은 변수가 있는지 확인해야 한다.

We should handle these variables differently as they would result in many dummy variables.
우리는 이러한 변수들이 많은 더미 변수를 초래할 수 있기 때문에 다르게 취급해야 한다.

In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    dist_values = train[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f,dist_values))

Only **ps_car_11_cat** has many distinct values, although it is still reasonable.
ps_car_11_cat만이 많은 뚜렷한 가치를 가지고 있지만, 여전히 합리적이다.

In [ ]:
# Script by https://www.kaggle.com/ogrellier
# Code: https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
train_encoded, test_encoded = target_encode(train["ps_car_11_cat"], 
                             test["ps_car_11_cat"], 
                             target=train.target, 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    
train['ps_car_11_cat_te'] = train_encoded
train.drop('ps_car_11_cat', axis=1, inplace=True)
meta.loc['ps_car_11_cat','keep'] = False  # Updating the meta
test['ps_car_11_cat_te'] = test_encoded
test.drop('ps_car_11_cat', axis=1, inplace=True)

**Exploratory Data Visualization**

Categorical variables

Let's look into the categorical variables and the proportion of customers with target = 1
범주형 변수와 target = 1인 고객 비율을 살펴봅시다.


In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index

for f in v:
    plt.figure()
    fig, ax = plt.subplots(figsize = (20,10))
    #Calculate the percentage of target = 1 per category value
    
    cat_perc = train[[f,'target']].groupby([f],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending = False, inplace = True)
    # Bar plot
    # Order the bars descending on target mean
    sns.barplot(ax = ax, x= f, y='target', data=cat_perc,order=cat_perc[f])
    plt.ylabel('% target', fontsize= 18)
    plt.xlabel(f, fontsize= 18)
    plt.tick_params(axis='both', which='major',labelsize=18)
    plt.show();

As we can see from the variables **with missing values**, it is a good idea to keep the missing values as a separate category value, instead of replacing them by the mode for instance. 
결측값이 있는 변수에서 알 수 있듯이, 결측값을 모드로 대체하는 대신 별도의 범주값으로 유지하는 것이 좋다. 

The customers with a missing value appear to have a much higher (in some cases much lower) probability to ask for an insurance claim.
결손가치가 있는 고객들은 보험금을 청구할 확률이 훨씬 더 높은 것 같다.


**Interval variables**
Checking the correlations between interval variables.
구간 변수 간의 상관 관계 확인.

A heatmap is a good way to visualize the correlation between variables. 
열 지도는 변수 간의 상관 관계를 시각화하는 좋은 방법이다. 

The code below is based on an example by Michael Waskom
아래 코드는 Michael Waskom의 예를 기반으로 한다

In [ ]:
def corr_heatmap(v):
    correlations = train[v].corr()
    
    # Create color map ranging between two colors
    
    cmap = sns.diverging_palette(220,10,as_cmap=True)
    
    fig, ax = plt.subplots(figsize = (10,10))
    sns.heatmap(correlations, cmap = cmap, vmax=1.0, center = 0, fmt='.2f',
               square=True, linewidths = .5, annot = True,
               cbar_kws={'shrink':.75})
    plt.show();
v = meta[(meta.level=='interval') & (meta.keep)].index
corr_heatmap(v)

There are a strong correlations between the variables:

* ps_reg_02 and ps_reg_03 (0.7)
* ps_car_12 and ps_car13 (0.67)
* ps_car_12 and ps_car14 (0.58)
* ps_car_13 and ps_car15 (0.67)
Seaborn has some handy plots to visualize the (linear) relationship between variables.
Seaborn은 변수 간의 (선형) 관계를 시각화하기 위한 몇 가지 편리한 그림을 가지고 있다.

We could use a pairplot to visualize the relationship between the variables.
우리는 변수 사이의 관계를 시각화하기 위해 쌍 그림을 사용할 수 있다. 

But because the heatmap already showed the limited number of correlated variables, we'll look at each of the highly correlated variables separately.
그러나 열 지도는 이미 상관 변수의 제한된 수를 보여 주었기 때문에, 각각의 높은 상관 변수를 별도로 살펴볼 것이다.

NOTE: I take a sample of the train data to speed up the process. 
주: 속도를 높이기 위해 train 데이터의 샘플을 채취한다.

In [ ]:
s = train.sample(frac=0.1)

ps_reg_02 and ps_reg_03

As the regression line shows, there is a linear relationship between these variables. 
회귀선이 보여주는 것처럼, 이 변수들 사이에는 선형 관계가 있다.

Thanks to the hue parameter we can see that the regression lines for target=0 and target=1 are the same.
 hue 파라미터 덕분에 target=0과 target=1에 대한 회귀선이 동일하다는 것을 알 수 있다.


In [ ]:
sns.lmplot(x='ps_reg_02', y = 'ps_reg_03', data=s, hue='target',palette = 'Set1',scatter_kws={'alpha':0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_12', y ='ps_car_13', data=s, hue='target',palette='Set1', scatter_kws={'alpha':0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_12',y='ps_car_14',data=s,hue='target',palette='Set1',scatter_kws={'alpha':0.3})
plt.show()

In [ ]:
sns.lmplot(x='ps_car_15',y='ps_car_13',data=s,hue='target',palette='Set1',scatter_kws={'alpha':0.3})
plt.show()

Allright, so now what? How can we decide which of the correlated variables to keep?
어떤 상관 변수를 유지할 것인지 어떻게 결정할 수 있는가?

We could perform Principal Component Analysis (PCA) on the variables to reduce the dimensions.
우리는 차수를 줄이기 위해 변수에 대한 주성분분석(PCA)을 수행할 수 있다.

In the AllState Claims Severity Competition I made this kernel to do that.
올스테이트는 심각성을 주장하지만, 나는 그것을 하기 위해 이 커널을 만들었다.

But as the number of correlated variables is rather low, we will let the model do the heavy-lifting.
그러나 상관 변수의 수가 상대적으로 적기 때문에 우리는 모델이 무거운 것을 들도록 할 것이다.



**Checking the correlations between ordinal variables**

In [ ]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
corr_heatmap(v)

For the ordinal variables we do not see many correlations. 
순서형 변수에 대해서는 많은 상관관계가 보이지 않는다. 

We could, on the other hand, look at how the distributions are when grouping by the target value.
반면에 목표값을 기준으로 그룹화할 때 분포가 어떤지 살펴볼 수 있다.


In [ ]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
print('Before dummification we have {} variables in train'.format(train.shape[1]))
train = pd.get_dummies(train,columns=v, drop_first = True)
print('After dummification we have {} variables in train'.format(train.shape[1]))

So, creating dummy variables adds 52 variables to the training set.
따라서 더미 변수를 만드는 것은 52개의 변수를 훈련 세트에 추가한다.

**Creating interaction variables**

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
poly = PolynomialFeatures(degree=2, interaction_only=False,include_bias=False)
interactions = pd.DataFrame(data=poly.fit_transform(train[v]),columns=poly.get_feature_names(v))
interactions.drop(v, axis=1,inplace=True)#Remove the original columns
#concat the interaction variables to the train data
print('Before creating interactions we have {} variables in train'.format(train.shape[1]))
train = pd.concat([train,interactions],axis=1)
print('After creating interactions we have {} variables in train'.format(train.shape[1]))

This adds extra interaction variables to the train data. 
이것은 train 데이터에 추가적인 상호작용 변수를 추가한다. 

Thanks to the get_feature_names method we can assign column names to these new variables.
get_feature_names 메서드 덕분에 이 새 변수에 열 이름을 할당할 수 있다.

**Feature selection**

**Removing features with low or zero variance**

Personally, I prefer to let the classifier algorithm chose which features to keep.
개인적으로, 나는 분류기 알고리즘이 어떤 특징을 유지할지를 선택하도록 하는 것을 선호한다. 

But there is one thing that we can do ourselves.
하지만 우리가 스스로 할 수 있는 일이 한 가지 있다.

That is removing features with no or a very low variance. 
그것은 분산이 없거나 매우 낮은 형상을 제거하는 것이다. 

Sklearn has a handy method to do that: VarianceThreshold. 
Sklearn은 VariantThreshold라는 유용한 방법을 가지고 있다.

By default it removes features with zero variance.
기본적으로 분산이 0인 형상을 제거한다.

This will not be applicable for this competition as we saw there are no zero-variance variables in the previous steps.
이전 단계에서 분산 변수가 없는 것을 보았으므로 이 경기에는 적용되지 않을 것이다. 

But if we would remove features with less than 1% variance, we would remove 31 variables.
그러나 1% 미만의 차이를 가진 형상을 제거하면 31개의 변수를 제거할 것이다.


In [ ]:
selector = VarianceThreshold(threshold=.01)
selector.fit(train.drop(['id','target'],axis=1)) #fit to train without id and target variables

f = np.vectorize(lambda x : not x) # Function to toggle boolean array elements

v = train.drop(['id','target'], axis=1).columns[f(selector.get_support())]
print('{} variables have too low variance.'.format(len(v)))
print('These variables are {}'.format(list(v)))

We would lose rather many variables if we would select based on variance. 만약 우리가 분산에 근거하여 선택한다면 우리는 오히려 많은 변수를 잃게 될 것이다. 

But because we do not have so many variables, we'll let the classifier chose.
하지만 우리는 변수가 많지 않기 때문에 분류자를 선택하게 될 것이다. 

For data sets with many more variables this could reduce the processing time
변수가 더 많은 데이터 세트의 경우 처리 시간을 줄일 수 있다.



Sklearn also comes with other feature selection methods. 
sklearn은 또한 다른 기능 선택 방법과 함께 제공된다.

One of these methods is SelectFromModel in which you let another classifier select the best features and continue with these.
이러한 방법 중 하나는 SelectFromModel로, 당신은 다른 분류자가 가장 좋은 특징을 선택하고 이것들을 계속하도록 한다.

Below I'll show you how to do that with a Random Forest.
아래에서는 랜덤 포레스트로 어떻게 하는지 알려줄게.

**Selecting features with a Random Forest and SelectFromModel**

Here we'll base feature selection on the feature importances of a random forest. 
여기서는 랜덤 포리스트의 피쳐 선택을 기반으로 한다.

With Sklearn's SelectFromModel you can then specify how many variables you want to keep.
Sklearn's SelectFromModel을 사용하여 보관할 변수 수를 지정할 수 있다. 

You can set a threshold on the level of feature importance manually.
형상 중요도 수준에 대한 임계값을 수동으로 설정할 수 있다. 

But we'll simply select the top 50% best variables.
그러나 우리는 단지 상위 50%의 변수를 선택할 것이다.


In [ ]:
X_train= train.drop(['id','target'], axis=1)
y_train = train['target']

feat_labels = X_train.columns

rf = RandomForestClassifier(n_estimators=1000,random_state=0,n_jobs=-1)

rf.fit(X_train, y_train)
importances = rf.feature_importances_

indices = np.argsort(rf.feature_importances_)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f"%(f+1,30,feat_labels[indices[f]],importances[indices[f]]))

With SelectFromModel we can specify which prefit classifier to use and what the threshold is for the feature importances. 
SelectFromModel을 사용하여 사용할 프리핏 분류기 및 피쳐가져오기에 대한 임계값을 지정할 수 있다. 

With the get_support method we can then limit the number of variables in the train data.
get_support 방법을 사용하여 train 데이터의 변수 수를 제한할 수 있다.

In [ ]:
sfm = SelectFromModel(rf, threshold='median', prefit=True)
print('Number of features before selection: {}'.format(X_train.shape[1]))
n_features = sfm.transform(X_train).shape[1]
print('Number of features after selection: {}'.format(n_features))
selected_vars = list(feat_labels[sfm.get_support()])

In [ ]:
train = train[selected_vars + ['target']]

**Feature scaling**

As mentioned before, we can apply standard scaling to the training data.
앞에서 언급한 바와 같이 표준 스케일링을 training 데이터에 적용할 수 있다.

Some classifiers perform better when this is done.
어떤 분류자들은 이것이 끝났을 때 더 잘 한다.



In [ ]:
scaler = StandardScaler()
scaler.fit_transform(train.drop(['target'], axis=1))

**Conclusion**

